In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357247 sha256=5bd9974531be7b42b08d9380b41d0fd3844c43f3b2ef94aa29fc351d81279f20
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
# Let's import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from surprise import Dataset, Reader, SVD  # -- !pip install surprise
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Let's import dataset
df = pd.read_excel('/content/Music_recommendation.xlsx')

In [4]:
df.head()

,Listener_id,name,artist,spotify_id,preview,img,danceability,energy,loudness,speechiness,...,liveness,valence,acousticness_artist,danceability_artist,energy_artist,instrumentalness_artist,liveness_artist,speechiness_artist,valence_artist,Streams
0,861,Mood (feat. iann dior),24kGoldn,3tjFYV6RSFtuktYl3ZtYcq,https://p.scdn.co/mp3-preview/45cb08fdb67744ab...,https://i.scdn.co/image/ab67616d0000b273ff8c98...,0.700,0.722,-3.558000e+16,0.0369,...,0.2720,0.756,0.118269,0.731588,0.681235,1.630000e+09,0.160000,0.123765,0.566824,1538727
1,3773,Dynamite,BTS,0t1kP63rueHleOhQkYSXFY,https://p.scdn.co/mp3-preview/a707728846c105f4...,https://i.scdn.co/image/ab67616d0000b273755995...,0.746,0.765,-4.410000e+00,0.0993,...,0.0936,0.737,0.099350,0.614798,0.786452,4.370000e+10,0.202608,0.128515,0.533369,2826709
2,3093,HawÃ¡i,Maluma,4uoR6qeWeuL4Qeu2qJzkuG,https://p.scdn.co/mp3-preview/07037d916c2e4ea2...,https://i.scdn.co/image/ab67616d0000b273480552...,0.787,0.728,-3.455000e+03,0.3890,...,0.0629,0.545,0.200861,0.741490,0.759686,8.850000e+10,0.156502,0.120922,0.707190,7296275
3,467,Savage Love (Laxed - Siren Beat),Jawsh 685,1xQ6trAsedVPCdbtDAmk0c,https://p.scdn.co/mp3-preview/d709526bcdffa668...,https://i.scdn.co/image/ab67616d0000b273e3eb3b...,0.767,0.481,-8.520000e+00,0.0803,...,0.2690,0.761,0.116467,0.746333,0.573667,4.613333e-01,0.301667,0.100100,0.893000,4419278
4,4427,Head & Heart (feat. MNEK),Joel Corry,6cx06DFPPHchuUAcTxznu9,https://p.scdn.co/mp3-preview/a860b3fe04f48053...,https://i.scdn.co/image/ab67616d0000b27391e93c...,0.734,0.874,-3.158000e+03,0.0662,...,0.0489,0.905,0.094600,0.697000,0.921000,1.900000e-05,0.159000,0.042400,0.722000,8699502


## 1. Data Preprocessing

In [5]:
# Let's do some preprocessing
df.head()

,Listener_id,name,artist,spotify_id,preview,img,danceability,energy,loudness,speechiness,...,liveness,valence,acousticness_artist,danceability_artist,energy_artist,instrumentalness_artist,liveness_artist,speechiness_artist,valence_artist,Streams
0,861,Mood (feat. iann dior),24kGoldn,3tjFYV6RSFtuktYl3ZtYcq,https://p.scdn.co/mp3-preview/45cb08fdb67744ab...,https://i.scdn.co/image/ab67616d0000b273ff8c98...,0.700,0.722,-3.558000e+16,0.0369,...,0.2720,0.756,0.118269,0.731588,0.681235,1.630000e+09,0.160000,0.123765,0.566824,1538727
1,3773,Dynamite,BTS,0t1kP63rueHleOhQkYSXFY,https://p.scdn.co/mp3-preview/a707728846c105f4...,https://i.scdn.co/image/ab67616d0000b273755995...,0.746,0.765,-4.410000e+00,0.0993,...,0.0936,0.737,0.099350,0.614798,0.786452,4.370000e+10,0.202608,0.128515,0.533369,2826709
2,3093,HawÃ¡i,Maluma,4uoR6qeWeuL4Qeu2qJzkuG,https://p.scdn.co/mp3-preview/07037d916c2e4ea2...,https://i.scdn.co/image/ab67616d0000b273480552...,0.787,0.728,-3.455000e+03,0.3890,...,0.0629,0.545,0.200861,0.741490,0.759686,8.850000e+10,0.156502,0.120922,0.707190,7296275
3,467,Savage Love (Laxed - Siren Beat),Jawsh 685,1xQ6trAsedVPCdbtDAmk0c,https://p.scdn.co/mp3-preview/d709526bcdffa668...,https://i.scdn.co/image/ab67616d0000b273e3eb3b...,0.767,0.481,-8.520000e+00,0.0803,...,0.2690,0.761,0.116467,0.746333,0.573667,4.613333e-01,0.301667,0.100100,0.893000,4419278
4,4427,Head & Heart (feat. MNEK),Joel Corry,6cx06DFPPHchuUAcTxznu9,https://p.scdn.co/mp3-preview/a860b3fe04f48053...,https://i.scdn.co/image/ab67616d0000b27391e93c...,0.734,0.874,-3.158000e+03,0.0662,...,0.0489,0.905,0.094600,0.697000,0.921000,1.900000e-05,0.159000,0.042400,0.722000,8699502


In [6]:
# Let's remove unnecessary columns
del df['preview']
del df['img']
del df['spotify_id']

In [7]:
# Let's check up
df.dtypes

Listener_id                  int64
name                        object
artist                      object
danceability               float64
energy                     float64
loudness                   float64
speechiness                float64
acousticness               float64
instrumentalness           float64
liveness                   float64
valence                    float64
acousticness_artist        float64
danceability_artist        float64
energy_artist              float64
instrumentalness_artist    float64
liveness_artist            float64
speechiness_artist         float64
valence_artist             float64
Streams                      int64
dtype: object

###### Great ✅

In [8]:
# Convert all entries to strings
df['name'] = df['name'].astype(str)
df['artist'] = df['artist'].astype(str)

# Initialize the label encoders
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Fit and transform the 'name' and 'artist' columns
df['name_encoded'] = label_encoder1.fit_transform(df['name'])
df['artist_encoded'] = label_encoder2.fit_transform(df['artist'])

# Create dictionaries mapping original labels to encoded values
label_name_mapping = {index: label for index, label in enumerate(label_encoder1.classes_)}
label_artist_mapping = {index: label for index, label in enumerate(label_encoder2.classes_)}

# Optional: Save the dictionaries as JSON files
import json
with open('label_name_mapping.json', 'w') as json_file:
    json.dump(label_name_mapping, json_file)
with open('label_artist_mapping.json', 'w') as json_file:
    json.dump(label_artist_mapping, json_file)

In [ ]:
# Let's remove object columns
del df['name']
del df['artist']

In [ ]:
# Let's check up scale of features
for i in df.columns:
    print(f'• Min-Max of {i}:')
    print('Min:',df[i].min(),'| Max:',df[i].max())

• Min-Max of Listener_id:
Min: 1 | Max: 4999
• Min-Max of danceability:
Min: 0.0 | Max: 0.986
• Min-Max of energy:
Min: 2.01e-05 | Max: 322000000000.0
• Min-Max of loudness:
Min: -7.99799999999999e+16 | Max: 0.522
• Min-Max of speechiness:
Min: 0.0 | Max: 0.918
• Min-Max of acousticness:
Min: 0.0 | Max: 584000000000.0
• Min-Max of instrumentalness:
Min: 0.0 | Max: 609000000000.0
• Min-Max of liveness:
Min: 0.0 | Max: 0.993
• Min-Max of valence:
Min: 0.0 | Max: 0.991
• Min-Max of acousticness_artist:
Min: 0.0 | Max: 584000000000.0
• Min-Max of danceability_artist:
Min: 0.0 | Max: 0.978
• Min-Max of energy_artist:
Min: 2.01e-05 | Max: 203000000000.0
• Min-Max of instrumentalness_artist:
Min: 0.0 | Max: 607000000000.0
• Min-Max of liveness_artist:
Min: 0.0 | Max: 0.989
• Min-Max of speechiness_artist:
Min: 0.0 | Max: 0.913
• Min-Max of valence_artist:
Min: 0.0 | Max: 0.991
• Min-Max of Streams:
Min: 943 | Max: 9998043
• Min-Max of name_encoded:
Min: 0 | Max: 17126
• Min-Max of artist_enco

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Assuming df is already defined and contains the columns to be scaled
for col in ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence',
       'acousticness_artist', 'danceability_artist', 'energy_artist',
       'instrumentalness_artist', 'liveness_artist', 'speechiness_artist',
       'valence_artist', 'Streams']:
    df[col] = scaler.fit_transform(df[[col]])

In [ ]:
df.to_excel('Music_recommendation_edit.xlsx', index = False)

### First Method: Nearest Neighbors

In [ ]:
# Create the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=5, algorithm='auto')

In [ ]:
# Fit the model on the relevant features
columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness',
                    'acousticness', 'instrumentalness', 'liveness', 'valence',
                    'acousticness_artist', 'danceability_artist', 'energy_artist',
                    'instrumentalness_artist', 'liveness_artist', 'speechiness_artist',
                    'valence_artist', 'Streams']

nn_model.fit(df[columns_to_scale])

# Function to get recommendations
def get_recommendations(listener_id, df, model, n_recommendations=5):
    listener_data = df[df['Listener_id'] == listener_id][columns_to_scale]
    distances, indices = model.kneighbors(listener_data, n_neighbors=n_recommendations)

    recommended_listeners = df.iloc[indices[0]]['Listener_id'].values
    recommended_musics_label = df.iloc[indices[0]]['name_encoded'].values
    recommended_artists_label = df.iloc[indices[0]]['artist_encoded'].values

    recommended_musics = [label_name_mapping[i] for i in recommended_musics_label]
    recommended_artists = [label_artist_mapping[i] for i in recommended_artists_label]

    recommendations = {
        'Recommended Listeners': recommended_listeners,
        'Recommended Music': recommended_musics,
        'Recommended Artists': recommended_artists
    }

    return recommendations

In [ ]:
# Example usage
cont = True
while cont:
    listener_id = int(input('• Please enter listener id:'))
    recommendations = get_recommendations(listener_id, df, nn_model)
    print("")
    for i,y in recommendations.items():
        print(f'• {i}')
        print(f'    {set(y)}')
    cont_answ = input('• Do you want to continue analyze? (True/False):')
    if cont_answ.lower() == "true":
        print(' - It is amazing analyze 😉👌!')
        cont = True
    elif cont_answ.lower() == "false":
        print(' - Thanks for using 😊!')
        cont = False

• Please enter listener id:12

• Recommended Listeners
    {3359, 12, 4082, 1119, 1919}
• Recommended Music
    {'Set Sail', 'Take It Off'}
• Recommended Artists
    {'Kesha', 'The Movement'}
• Do you want to continue analyze? (True/False):fALSE
 - Thanks for using 😊!


### Second Method: Guide to Using SVD for Recommendations

In [ ]:
# Only use Listener_id, name_encoded (as music ID), and Streams (as rating)
df_surprise = df[['Listener_id', 'name_encoded', 'Streams']]

In [ ]:
# Define a reader with the appropriate rating scale
reader = Reader(rating_scale=(df_surprise['Streams'].min(), df_surprise['Streams'].max()))

# Load the data from the DataFrame
data = Dataset.load_from_df(df_surprise, reader)

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD algorithm
algo = SVD()

# Train the algorithm on the training set
algo.fit(trainset)

# Evaluate the algorithm on the test set
predictions = algo.test(testset)
accuracy.rmse(predictions)

# Function to get top N recommendations
def get_top_n_recommendations(algo, user_id, df, n=5):
    # Get all items (music IDs) that the user hasn't rated yet
    all_music_ids = df['name_encoded'].unique()
    rated_music_ids = df[df['Listener_id'] == user_id]['name_encoded'].unique()
    unrated_music_ids = [mid for mid in all_music_ids if mid not in rated_music_ids]

    # Predict ratings for all unrated music
    predictions = [algo.predict(user_id, mid) for mid in unrated_music_ids]

    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top N recommended music IDs
    top_n_music_ids = [pred.iid for pred in predictions[:n]]

    # Get the corresponding artist IDs and music names
    top_n_artists = df[df['name_encoded'].isin(top_n_music_ids)]['artist_encoded'].unique()

    recommended_musics = [label_name_mapping[i] for i in top_n_music_ids]
    recommended_artists = [label_artist_mapping[i] for i in top_n_artists]


    recommendations = {
        'recommended_music': recommended_musics,
        'recommended_artists': recommended_artists
    }

    return recommendations

RMSE: 0.2850


In [ ]:
# Example usage
cont = True
while cont:
    listener_id = int(input('• Please enter listener id:'))
    recommendations = get_top_n_recommendations(algo, listener_id, df)
    for i,y in recommendations.items():
        print(f'• {i}')
        print(f'    {set(y)}')
    cont_answ = input('• Do you want to continue analyze? (True/False):')
    if cont_answ.lower() == "true":
        print(' - It is amazing analyze 😉👌!')
        cont = True
    elif cont_answ.lower() == "false":
        print(' - Thanks for using 😊!')
        cont = False

• Please enter listener id:12
• recommended_music
    {'Sing About It', 'Nice to Know You', 'Paranoid Android', 'çª\x81ç„¶å¥½æƒ³ä½\xa0', "03' Adolescence"}
• recommended_artists
    {'Radiohead', 'The Wood Brothers', 'Mayday', 'Incubus', 'J. Cole'}
• Do you want to continue analyze? (True/False):false
 - Thanks for using 😊!


### Third Method: Content-Based Filtering with Cosine Similarity

In [3]:
# Calculate the cosine similarity matrix
feature_columns = ['danceability', 'energy', 'loudness', 'speechiness',
                   'acousticness', 'instrumentalness', 'liveness', 'valence',
                   'acousticness_artist', 'danceability_artist', 'energy_artist',
                   'instrumentalness_artist', 'liveness_artist', 'speechiness_artist',
                   'valence_artist']

cosine_sim = cosine_similarity(df[feature_columns])

# Function to get recommendations based on content similarity
def get_content_based_recommendations(listener_id, df, cosine_sim, n_recommendations=5):
    listener_data = df[df['Listener_id'] == listener_id]
    listened_music_ids = listener_data['name_encoded'].values
    similarity_scores = cosine_sim[listener_data.index].mean(axis=0)
    similarity_scores_indices = similarity_scores.argsort()[::-1]
    recommendations = []
    for idx in similarity_scores_indices:
        music_id = df.iloc[idx]['name_encoded']
        if music_id not in listened_music_ids:
            recommendations.append(music_id)
        if len(recommendations) >= n_recommendations:
            break
    recommended_artists = df[df['name_encoded'].isin(recommendations)]['artist_encoded'].unique()

    recommended_musics = [label_name_mapping[i] for i in recommendations]
    recommended_artists = [label_artist_mapping[i] for i in recommended_artists]

    recommendation_results = {
        'recommended_music': recommended_musics,
        'recommended_artists': recommended_artists
    }

    return recommendation_results

In [4]:
# Example usage
cont = True
while cont:
    listener_id = int(input('• Please enter listener id:'))
    recommendations = get_content_based_recommendations(listener_id, df, cosine_sim)
    for i,y in recommendations.items():
        print(f'• {i}')
        print(f'    {set(y)}')
    cont_answ = input('• Do you want to continue analyze? (True/False):')
    if cont_answ.lower() == "true":
        print(' - It is amazing analyze 😉👌!')
        cont = True
    elif cont_answ.lower() == "false":
        print(' - Thanks for using 😊!')
        cont = False

• Please enter listener id:5
• recommended_music
    {"My Boo - Hitman's Club Mix", "Don't Owe You a Thang", 'Yiken (Certified)'}
• recommended_artists
    {'Priceless Da Roc', 'Ghost Town DJs', 'Gary Clark Jr.'}
• Do you want to continue analyze? (True/False):FALSE
 - Thanks for using 😊!


# Let's deploy with gradio

In [1]:
# Let's import libraries
import gradio as gr
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from surprise import Dataset, Reader, SVD  # -- !pip install surprise
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity

# Let's import dataset
df = pd.read_excel('Music_recommendation.xlsx')

# Let's remove unnecessary columns
del df['preview']
del df['img']
del df['spotify_id']

# Convert all entries to strings
df['name'] = df['name'].astype(str)
df['artist'] = df['artist'].astype(str)

# Initialize the label encoders
label_encoder1 = LabelEncoder()
label_encoder2 = LabelEncoder()

# Fit and transform the 'name' and 'artist' columns
df['name_encoded'] = label_encoder1.fit_transform(df['name'])
df['artist_encoded'] = label_encoder2.fit_transform(df['artist'])

# Create dictionaries mapping original labels to encoded values
label_name_mapping = {index: label for index, label in enumerate(label_encoder1.classes_)}
label_artist_mapping = {index: label for index, label in enumerate(label_encoder2.classes_)}

# Let's remove object columns
del df['name']
del df['artist']

scaler = MinMaxScaler()
# Assuming df is already defined and contains the columns to be scaled
for col in ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence',
       'acousticness_artist', 'danceability_artist', 'energy_artist',
       'instrumentalness_artist', 'liveness_artist', 'speechiness_artist',
       'valence_artist', 'Streams']:
    df[col] = scaler.fit_transform(df[[col]])



# Scale the feature columns
feature_columns = ['danceability', 'energy', 'loudness', 'speechiness',
                   'acousticness', 'instrumentalness', 'liveness', 'valence',
                   'acousticness_artist', 'danceability_artist', 'energy_artist',
                   'instrumentalness_artist', 'liveness_artist', 'speechiness_artist',
                   'valence_artist', 'Streams']

# Calculate the cosine similarity matrix
cosine_sim = cosine_similarity(df[feature_columns])

# Function to get recommendations based on content similarity
def get_content_based_recommendations(listener_id, df, cosine_sim, n_recommendations=5):
    listener_data = df[df['Listener_id'] == listener_id]
    listened_music_ids = listener_data['name_encoded'].values
    similarity_scores = cosine_sim[listener_data.index].mean(axis=0)
    similarity_scores_indices = similarity_scores.argsort()[::-1]
    recommendations = []
    for idx in similarity_scores_indices:
        music_id = df.iloc[idx]['name_encoded']
        if music_id not in listened_music_ids:
            recommendations.append(music_id)
        if len(recommendations) >= n_recommendations:
            break
    recommended_artists = df[df['name_encoded'].isin(recommendations)]['artist_encoded'].unique()

    recommended_musics = [label_name_mapping[i] for i in recommendations]
    recommended_artists = [label_artist_mapping[i] for i in recommended_artists]

    recommended_musics = list(set(recommended_musics))
    recommended_artists = list(set(recommended_artists))

    recommendation_results = {
        'recommended_music': recommended_musics,
        'recommended_artists': recommended_artists
    }

    # Generate HTML content
    html_content = "<h2>Music Recommendations</h2>"
    html_content += "<h3>Recommended Songs:</h3><ul>"
    for music in recommended_musics:
        html_content += f"<li>{music}</li>"
    html_content += "</ul><h3>Recommended Artists:</h3><ul>"
    for artist in recommended_artists:
        html_content += f"<li>{artist}</li>"
    html_content += "</ul>"

    return html_content


# Define the Gradio interface
def recommend(listener_id):
    try:
        listener_id = int(listener_id)
        recommendations = get_content_based_recommendations(listener_id, df, cosine_sim)
        return recommendations
    except ValueError:
        return "<p>Invalid listener ID. Please enter a valid number.</p>"

iface = gr.Interface(fn=recommend,
                     inputs="text",
                     outputs=gr.HTML(),
                     title="Music Recommendation System",
                     description="Enter a listener ID to get music recommendations based on content similarity.")

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1d4f374943add194f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 🔚 The End